In [ ]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')
import elastic_rods, elastic_knots
import numpy as np, matplotlib.pyplot as plt, time, io, os

from helpers import *
from parametric_curves import *
import py_newton_optimizer

from linkage_vis import LinkageViewer as Viewer, CenterlineViewer
from tri_mesh_viewer import PointCloudViewer, PointCloudMesh

%load_ext autoreload
%autoreload 2

In [ ]:
import parallelism
parallelism.set_max_num_tbb_threads(1)

 - [Compute equilibria of **twist-free** knots](#sec:twist_free)
 - [Compute equilibria of **constant-link** knots](#sec:constant_link)

----
<a id='sec:twist_free'></a>
# Compute the equilibium state of a **twist-free** knotted elastic rod

### Select a knot

In [ ]:
# Load the centerline from file...
file = '../data/4_1-smooth.obj'

rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
centerline = read_nodes_from_file(file)  # supported formats: obj, txt

pr = define_periodic_rod(centerline, material)
rod_list = elastic_knots.PeriodicRodList([pr])

In [ ]:
# ...or generate a parametric knot
p, q, a, r = 2, 3, 2, 4
rod_radius = 0.2

material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
centerline = generate_curve(torus_knot(p=p, q=q, a=a, r=r), npts=102)
pr = define_periodic_rod(centerline, material)
rod_list = elastic_knots.PeriodicRodList([pr])

### Viewer

In [ ]:
view = Viewer(rod_list, width=1024, height=640)
view.show()

### Equilibrium solve

In [ ]:
def callback(problem, iteration):
    if iteration % 5 == 0:
        view.update()
        
optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
optimizerOptions.niter = 1000
optimizerOptions.gradTol = 1e-8
hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.contactStiffness = 1e3
problemOptions.dHat = 2*rod_radius

fixedVars = []   # all the degrees of freedom can be modified by the optimizer

In [ ]:
# Optimize
report = elastic_knots.compute_equilibrium(
    rod_list, problemOptions, optimizerOptions, 
    fixedVars=fixedVars,
    externalForces=np.zeros(rod_list.numDoF()),
    softConstraints=[],
    callback=callback,
    hessianShift=hessianShift
)
view.update()

### Save result to file

In [ ]:
from helpers import write_obj
file = '../data/output.obj'
write_obj(file, rod_list)

### Modal analysis

In [ ]:
from helpers import vibrational_analysis
fixedVars = []
problemOptions.projectContactHessianPSD = False
lambdas, modes = vibrational_analysis(rod_list, problemOptions, optimizerOptions, n_modes=10, fixedVars=fixedVars)

In [ ]:
# Visualize vibrational modes
import mode_viewer
ndof = rod_list.numDoF()
amplitude = pr.restLength() / 20
mview = mode_viewer.ModeViewer(rod_list, modes[0:ndof, :], lambdas, amplitude=amplitude, width=1024, height=640)
mview.show()

Select **Mode 7** to visualize the first mode associated to a non-zero eigenvalue

----
<a id='sec:constant_link'></a>
# Compute the equilibium state of a knotted elastic rod with **prescribed Link**

In [ ]:
import vis, matplotlib
from linkage_vis import CenterlineMesh

def update_material_frame(viewer, pr):
    dc = pr.rod.deformedConfiguration()
    frameArray = np.array([d.d2 for d in dc.materialFrame])
    vmin, vmax = 0, 1.4  # yellow frame
    frame = vis.fields.VectorField(viewer.mesh, frameArray, vmin=vmin, vmax=vmax, colormap=matplotlib.cm.jet, glyph=vis.fields.VectorGlyph.CYLINDER)
    viewer.update(mesh=CenterlineMesh(pr.rod), vectorField=frame)
    

def compute_calugareanu_quantities(pr):
    """
    Compute quantities appearing in Calugareanu's theorem:
    Lk + \Phi / 2\pi = Tw + Wr
    """
    Lk = pr.link()
    Phi = pr.openingAngle()   # in [0, 2pi]
    Tw = pr.twist()
    Wr = pr.writhe()
    
    return Lk, Phi, Tw, Wr


def callback(problem, iteration):
    if iteration % 5 == 0:
        update_material_frame(view_framed_mat, rod_list[0])
        view_framed.update()

In [ ]:
# Generate knot
p, q, a, r = 2, 3, 2, 4
rod_radius = 0.2

material = elastic_rods.RodMaterial('ellipse', 2000, 0.3, [rod_radius, rod_radius])
centerline = generate_curve(torus_knot(p=p, q=q, a=a, r=r), npts=202)
pr = define_periodic_rod(centerline, material)
dc = pr.rod.deformedConfiguration()
rod_list = elastic_knots.PeriodicRodList([pr])

In [ ]:
view_framed = Viewer(pr.rod, width=1024, height=640)
view_framed_mat = CenterlineViewer(pr.rod, superView=view_framed)

update_material_frame(view_framed_mat, pr)
view_framed.show()

The yellow material frame represents the orientation of the cross-section

In [ ]:
# Set the (generalized) linking number of the framed curve Lk + \Phi / 2\pi \in R^3
gen_link = -5.8

pr = set_generalized_link(pr, gen_link)
Lk_start, Phi_start, Tw_start, Wr_start = compute_calugareanu_quantities(pr)
rod_list = elastic_knots.PeriodicRodList(pr)

# Update the material frame in the viewer
update_material_frame(view_framed_mat, pr)
view_framed.update(mesh=rod_list)

In [ ]:
optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
optimizerOptions.niter = 1000
optimizerOptions.gradTol = 1e-8
hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

problemOptions = elastic_knots.ContactProblemOptions()
problemOptions.contactStiffness = 1e3
problemOptions.dHat = 2*rod_radius

fixedVars = [pr.thetaOffset(), pr.numDoF()-1]  # pin \theta_0 and \Theta (== totalOpeningAngle) to preserve the Link

In [ ]:
# Optimize
report = elastic_knots.compute_equilibrium(
    rod_list, problemOptions, optimizerOptions, 
    fixedVars=fixedVars,
    externalForces=np.zeros(rod_list.numDoF()),
    softConstraints=[],
    callback=callback,
    hessianShift=hessianShift
)
view_framed.update()

### Check Călugăreanu's Theorem

In [ ]:
pr = rod_list[0]
Lk_end, Phi_end, Tw_end, Wr_end = compute_calugareanu_quantities(pr)

In [ ]:
from prettytable import PrettyTable

RED, GREEN, END = '\033[91m', '\033[92m', '\033[0m'
rows = [
    ['Start', Lk_start, Phi_start/(2*np.pi), RED+str(Tw_start)+END, RED+str(Wr_start)+END, Lk_start + Phi_start/(2*np.pi), GREEN+str(Tw_start + Wr_start)+END], 
    ['End',   Lk_end,   Phi_end/(2*np.pi),   RED+str(Tw_end)+END,   RED+str(Wr_end)+END,   Lk_end + Phi_end/(2*np.pi),     GREEN+str(Tw_end + Wr_end)+END    ]
]
tab = PrettyTable(['', 'Link', 'Phi/2pi', 'Twist', 'Writhe', 'Link + Phi/2pi', 'Twist + Writhe'])
tab.float_format = '.3'
tab.add_rows(rows)
print(tab)

Twist gets traded for writhe during the optimization. Their sum stays constant.